In [50]:
#pip install NRCLex

In [51]:
# Import required modules
import nltk
nltk.download('punkt')
import pandas as pd
import numpy as np
from nltk import word_tokenize
from nltk.stem.snowball import SnowballStemmer
import requests
from LeXmo import LeXmo
import pickle
import os
from tqdm.notebook import tqdm
import statsmodels.formula.api as sm
from scipy.stats import t 

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ellietupin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [52]:
sp = "sp500"

In [53]:
df_path = "data/earning_call_dataframe/df_final_"+sp 
# df_path = "data/df_final"
with open(df_path,"rb") as file :
    df_final = pickle.load(file)
display(df_final)

,id,date,ticker,text,earning_call_quarter
0,4264085,2019-05-14,A,Question-and-Answer Session\n \nOperator\n \n[...,2019Q1
1,4285738,2019-08-14,A,Question-and-Answer Session\n \nOperator\n \n[...,2019Q2
2,4309111,2019-11-26,A,Question-and-Answer Session\n \nOperator\n \n[...,2019Q3
3,4325101,2020-02-18,A,Question-and-Answer Session\n \nOperator\n \n[...,2019Q4
4,4349475,2020-05-21,A,Question-and-Answer Session\n \nOperator\n \n[...,2020Q1
...,...,...,...,...,...
1855,4422752,2021-04-30,DRE,Question-and-Answer Session\n \nOperator\n \n[...,2021Q1
1856,4442846,2021-07-29,DRE,Question-and-Answer Session\n \nOperator\n \nT...,2021Q2
1857,4463265,2021-10-28,DRE,Question-and-Answer Session\n \nOperator\n \nT...,2021Q3
1858,4482287,2022-01-27,DRE,Question-and-Answer Session \n \nOperator\n \n...,2021Q4


# Import datas from yahoo finance

Import html tickers from data/ticker
dir_path = "data/ticker/"
list_tickers = os.listdir(dir_path)
list_tickers.remove('.DS_Store') # for mac
list_tickers.remove('id_to_date')
print(list_tickers)

Test with few stocks
list_tickers = ['AAPL']

In [54]:
# Get the month from which we import stock returns :
    # Q1 : from january (1) to march (3)
    # Q2 : from april (4) to june (6)
    # Q3 : from july (7) to septmber (9)
    # Q4 : from october (10) to december (12)

#df_final['earning_call_quarter'] = pd.to_datetime(df_final['earning_call_quarter'])
#df_final['date'] = pd.to_datetime(df_final['date'])

# Quarters start 1 end
quarter_start = df_final['earning_call_quarter'].iloc[0].quarter
quarter_end = df_final['earning_call_quarter'].iloc[-1].quarter 

# Months start 1 end
month_start = quarter_start*2-1
month_end = quarter_end*3

# Day end : 30 or 31 ?
if month_end in [3,12]:
    day_end = 31
else:
    day_end = 30


In [67]:
import pandas as pd
import yfinance as yf
import datetime
import time
import requests
import io

# Take a margin of 1 year to calculate abnormal returns
start = datetime.datetime(df_final['earning_call_quarter'].iloc[0].year,month_start,2)
end   = datetime.datetime(df_final['earning_call_quarter'].iloc[-1].year,month_end,day_end)

# Create empty dataframe
stock_final = pd.DataFrame()

# Iterate over each symbol
list_tickers = list(set(list(df_final['ticker'])))
for i in list_tickers:  
    
    # print the symbol which is being downloaded
    print( str(list_tickers.index(i)) + str(' : ') + i, sep=',', end=',', flush=True)  
    
    try:
        # download the stock price 
        stock = []
        stock = yf.download(i,start=start, end=end, progress=False)
        
        # append the individual stock prices 
        if len(stock) == 0:
            None
        else:
            stock['ticker']=i
            stock_final = stock_final.append(stock,sort=False)
    except Exception:
        None
        
display(stock_final)

0 : BALL,1 : AIG,2 : DLTR,3 : AVB,4 : CTSH,5 : CMA,6 : APD,7 : C,8 : BMY,9 : CDNS,10 : ANTM,11 : AOS,12 : ARE,13 : AXP,14 : CMG,15 : DRE,16 : CCI,17 : ALGN,18 : ALB,19 : AMP,20 : BXP,21 : A,22 : BA,23 : BIIB,24 : CLX,25 : AEE,26 : DIS,27 : ADP,28 : CBRE,29 : CHTR,30 : BDX,31 : CHD,32 : CI,33 : DFS,34 : AON,35 : CME,36 : AZO,37 : AIZ,38 : BEN,39 : BAC,40 : BBWI,41 : AAL,42 : DD,43 : ANSS,44 : ABT,45 : CBOE,46 : AME,47 : BLK,48 : AES,49 : AFL,50 : AMZN,51 : BK,52 : BKR,53 : CMS,54 : CE,55 : ALL,56 : CINF,57 : ANET,58 : BBY,59 : ALK,60 : DHI,61 : CHRW,62 : DE,63 : CAH,64 : AEP,65 : CMI,66 : ABBV,67 : APH,68 : ADM,69 : AMAT,70 : DGX,71 : ADSK,72 : CDW,73 : DHR,74 : BRO,75 : BKNG,76 : CAT,77 : AKAM,78 : CB,79 : COF,80 : D,81 : ABMD,82 : DLR,83 : DG,84 : APTV,85 : ATO,86 : CPT,87 : CDAY,88 : CSX,89 : AAPL,90 : AVY,91 : APA,92 : DAL,93 : DOV,94 : CF,95 : BR,96 : CL,97 : BSX,98 : CVS,99 : CZR,

,Open,High,Low,Close,Adj Close,Volume,ticker
Date,,,,,,,
2019-01-02,45.250000,45.349998,44.400002,44.669998,43.405777,3910500,BALL
2019-01-03,44.430000,45.029999,43.869999,44.259998,43.007381,3105100,BALL
2019-01-04,44.869999,46.070000,44.689999,45.590000,44.299740,2709700,BALL
2019-01-07,45.619999,47.119999,45.360001,46.439999,45.125679,2973300,BALL
2019-01-08,46.779999,47.349998,46.720001,46.910000,45.582386,3122800,BALL
...,...,...,...,...,...,...,...
2022-03-24,77.099998,79.010002,75.420998,78.989998,78.989998,2076300,CZR
2022-03-25,79.150002,79.150002,77.040001,77.400002,77.400002,1165600,CZR
2022-03-28,77.599998,78.500000,75.500000,76.879997,76.879997,1822700,CZR


In [68]:
# Add quarter column
stock_final['date']=stock_final.index
stock_final['quarter'] = stock_final['date'].dt.to_period('Q')
# Calculate market capitalization
stock_final['market_cap']=stock_final['Adj Close']*stock_final['Volume']
# Drop useless columns
stock_final=stock_final.drop(columns=['Open', 'High','Low','Close','date','Adj Close','Volume'])

# Sum of 100 stocks for each quarter
stock_final = stock_final.groupby(['quarter']).sum()
stock_final['percent_change']=stock_final['market_cap'].pct_change()

stock_final = stock_final.iloc[1:,:]  # remove first row
stock_final.index = stock_final.index.to_timestamp()
display(stock_final)

,market_cap,percent_change
quarter,,
2019-04-01,2.359978e+12,-0.053799
2019-07-01,2.220735e+12,-0.059002
2019-10-01,2.352196e+12,0.059197
2020-01-01,3.722451e+12,0.582543
2020-04-01,3.887711e+12,0.044395
2020-07-01,4.118104e+12,0.059262
2020-10-01,3.796674e+12,-0.078053
2021-01-01,3.833942e+12,0.009816
2021-04-01,3.513833e+12,-0.083493


# Text analysis

In [79]:
# Preprocessing
texts = df_final['text']
n = len(texts)
n = 15

In [80]:
#N=len(texts)

for i in tqdm(range(n)):
    # creating objects
    text=texts[i]
    emo = LeXmo.LeXmo(text)
    emo.pop('text', None)
    emo_array = np.array(list(emo.items()))
    score = emo_array[:,1]
    if i==0:
        con = score
    else :
        con = np.vstack((con, score))

  0%|          | 0/15 [00:00<?, ?it/s]

In [81]:
# Create dataframe
score = np.reshape(con, (n, 10))
column=['anger','anticipation','disgust','fear','joy','negative','positive','sadness','surprise','trust']
df = pd.DataFrame(score, columns = column)
df[column]=df[column].astype(float)

#number_words = np.zeros(n)
#for i in range(n):
#    number_words[i] = len(texts[i])
#df['number_words']=number_words
df['id'] = df_final['id']
df['quarter'] = df_final['earning_call_quarter']
df['ticker'] = df_final['ticker']
df

,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust,id,quarter,ticker
0,0.002573,0.015214,0.001566,0.001678,0.006377,0.008166,0.038819,0.003020,0.007048,0.022374,4264085,2019Q1,A
1,0.002168,0.017456,0.001255,0.001255,0.010724,0.005819,0.039247,0.002054,0.008329,0.022476,4285738,2019Q2,A
2,0.002148,0.017976,0.000678,0.001357,0.013341,0.006670,0.047824,0.002826,0.008027,0.025664,4309111,2019Q3,A
3,0.002380,0.019517,0.001428,0.002975,0.009401,0.007973,0.039034,0.003213,0.010115,0.019279,4325101,2019Q4,A
4,0.004648,0.012218,0.001594,0.002789,0.008367,0.009960,0.036255,0.003453,0.005179,0.018459,4349475,2020Q1,A
5,0.003350,0.016287,0.000578,0.001964,0.010050,0.007508,0.038581,0.003234,0.006700,0.017905,4369599,2020Q2,A
6,0.002066,0.013978,0.001459,0.001094,0.011183,0.008752,0.044488,0.003525,0.007536,0.023095,4391129,2020Q3,A
7,0.001496,0.013811,0.000806,0.001151,0.010818,0.005985,0.040396,0.002647,0.006445,0.021291,4406522,2020Q4,A
8,0.002315,0.017873,0.000900,0.001929,0.011701,0.005915,0.047705,0.003215,0.006558,0.020573,4431288,2021Q1,A
9,0.001331,0.015846,0.000242,0.001089,0.008951,0.006169,0.037861,0.003145,0.007500,0.017177,4450247,2021Q2,A


In [82]:
df_change = df
#column_change=['anger_change','anticipation_change','disgust_change','fear_change','joy_change','negative_change','positive_change','sadness_change','surprise_change','trust_change']
df_change[column] = df[column].pct_change()
#df_change.columns =column_change
#df_change = df_change.drop(columns=['id'])
df_change = df_change.groupby(['quarter']).mean()
df_change=df_change.iloc[1:,:]  # remove first row
df_change = df_change.fillna(0) # Remmplace Nan with zeros
df_change.index = (df_change.index).to_timestamp()
display(df_change)

# Merge dataframes
df_4 = pd.merge(stock_final, df_change, left_index=True, right_index=True)
# Drop first row
#df_change=df_change.drop(df_change.groupby('ticker').head(1).index, axis=0)
display(df_4)

,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust
quarter,,,,,,,,,,
2019-04-01,-0.026106,0.172643,0.151611,-0.324919,0.377327,-0.083514,-0.022175,-0.103786,0.261362,0.064781
2019-07-01,-0.009045,0.029816,-0.459479,0.081042,0.243965,0.146399,0.218529,0.376327,-0.036194,0.141862
2019-10-01,0.108000,0.085701,1.105201,1.192917,-0.295293,0.195326,-0.183799,0.136808,0.260155,-0.248805
2020-01-01,0.952888,-0.373987,0.115936,-0.062614,-0.110076,0.249182,-0.071187,0.074605,-0.487982,-0.042499
2020-04-01,-0.279295,0.333088,-0.637577,-0.295863,0.201175,-0.246159,0.064171,-0.063275,0.293573,-0.030061
2020-07-01,-0.383154,-0.141768,1.525441,-0.442918,0.112742,0.165588,0.153083,0.089848,0.124837,0.289876
2020-10-01,-0.275954,-0.012004,-0.447683,0.052033,-0.032587,-0.316179,-0.091975,-0.249066,-0.144799,-0.078087
2021-01-01,0.546988,0.294170,0.117269,0.675903,0.081611,-0.011647,0.180931,0.214423,0.017513,-0.033713
2021-04-01,-0.425111,-0.113414,-0.731221,-0.435563,-0.235012,0.042981,-0.206340,-0.021643,0.143630,-0.165104


,market_cap,percent_change,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust
quarter,,,,,,,,,,,,
2019-04-01,2.359978e+12,-0.053799,-0.026106,0.172643,0.151611,-0.324919,0.377327,-0.083514,-0.022175,-0.103786,0.261362,0.064781
2019-07-01,2.220735e+12,-0.059002,-0.009045,0.029816,-0.459479,0.081042,0.243965,0.146399,0.218529,0.376327,-0.036194,0.141862
2019-10-01,2.352196e+12,0.059197,0.108000,0.085701,1.105201,1.192917,-0.295293,0.195326,-0.183799,0.136808,0.260155,-0.248805
2020-01-01,3.722451e+12,0.582543,0.952888,-0.373987,0.115936,-0.062614,-0.110076,0.249182,-0.071187,0.074605,-0.487982,-0.042499
2020-04-01,3.887711e+12,0.044395,-0.279295,0.333088,-0.637577,-0.295863,0.201175,-0.246159,0.064171,-0.063275,0.293573,-0.030061
2020-07-01,4.118104e+12,0.059262,-0.383154,-0.141768,1.525441,-0.442918,0.112742,0.165588,0.153083,0.089848,0.124837,0.289876
2020-10-01,3.796674e+12,-0.078053,-0.275954,-0.012004,-0.447683,0.052033,-0.032587,-0.316179,-0.091975,-0.249066,-0.144799,-0.078087
2021-01-01,3.833942e+12,0.009816,0.546988,0.294170,0.117269,0.675903,0.081611,-0.011647,0.180931,0.214423,0.017513,-0.033713
2021-04-01,3.513833e+12,-0.083493,-0.425111,-0.113414,-0.731221,-0.435563,-0.235012,0.042981,-0.206340,-0.021643,0.143630,-0.165104


# OLS

In [83]:
# ols
result = sm.ols(formula="percent_change ~ anger+anticipation+disgust+fear+joy+negative+positive+sadness+surprise+trust", data=df_4).fit()
#anger+anticipation+disgust+fear+joy+negative+positive+sadness+surprise+trust
# ATTENTION NAN DANS DiSGUST
result.summary()

/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=12
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         percent_change   R-squared:                       0.803
Model:                            OLS   Adj. R-squared:                 -1.164
Method:                 Least Squares   F-statistic:                    0.4083
Date:                Wed, 15 Jun 2022   Prob (F-statistic):              0.851
Time:                        23:53:43   Log-Likelihood:                 13.929
No. Observations:                  12   AIC:                            -5.859
Df Residuals:                       1   BIC:                           -0.5247
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        0.0325      0.146      0.223      0.860      -1.822       1.887
anger            0.6153      1.111      0.554      0.678     -13.496      14.727
anticipation    -0.6866      2.402     -0.286      0.823     -31.210      29.837
disgust          0.0017      0.294      0.006      0.996      -3.735       3.739
fear            -0.1398      0.355     -0.393      0.761      -4.656       4.376
joy             -0.3482      3.040     -0.115      0.927     -38.978      38.281
negative        -0.7601      2.900     -0.262      0.837     -37.612      36.092
positive        -0.3153      6.191     -0.051      0.968     -78.983      78.353
sadness          0.4729      2.168      0.218      0.863     -27.070      28.016
surprise         0.5670      1.112      0.510      0.700     -13.558      14.692
trust            0.4831      8.813      0.055      0.965    -111.500     112.466
==============================================================================
Omnibus:                        1.302   Durbin-Watson:                   0.575
Prob(Omnibus):                  0.522   Jarque-Bera (JB):                1.025
Skew:                           0.563   Prob(JB):                        0.599
Kurtosis:                       2.117   Cond. No.                         190.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""